#### --coding: utf-8--

## Import the data

In [1]:
import Tkinter
import tkFileDialog
root = Tkinter.Tk()
filename = tkFileDialog.askopenfilename(parent=root, initialdir='/users/nikitaskoussis/Dropbox/Science/PVB304/Article/MNE', 
                                    title='Please select your EEG file')
root.destroy()

## Import the relevant libraries

In [5]:
import mne
import scipy
import numpy 
import matplotlib
matplotlib.use('Qt5Agg')


## Read the raw file

In [112]:
raw = mne.io.read_raw_eeglab(filename)
print(raw)
print(raw.info)

Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file


<ipython-input-112-c48fac2ac63b>:1: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = mne.io.read_raw_eeglab(filename)


<RawEEGLAB  |  None, n_channels x n_times : 258 x 107996 (432.0 sec), ~213.1 MB, data loaded>
<Info | 16 non-empty fields
    bads : list | 0 items
    buffer_size_sec : float | 1.0
    ch_names : list | E1, E2, E3, E4, E5, E6, E7, E8, E9, ...
    chs : list | 258 items (EEG: 257, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : 'mne.transforms.Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 125.0 Hz
    meas_date : numpy.ndarray | 1970-01-01 10:00:00
    nchan : int | 258
    projs : list | 0 items
    sfreq : float | 250.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    pro

In [5]:
print(raw.ch_names)

[u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E82', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E102', u'E103', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E111', u'E112', u'E113', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E120', u'E121', u'E122', u'E123', 

### Preprocessing for speed (may not be needed)

In [78]:
#tmin, tmax = 20, 200
#raw.crop(tmin, tmax).load_data()  # 20s to 200s data segment for speed
#raw.plot(show_options=True) #plot of raw data

<RawEEGLAB  |  None, n_channels x n_times : 258 x 45001 (180.0 sec), ~89.1 MB, data loaded>

### Designating bad channels

In [113]:
# bad channels
raw.info['bads'] = ['REF','E82','E102','E103','E111','E112','E113','E120','E121',
                    'E122','E123','E124','E133','E134','E135','E136','E145','E146',
                    'E147','E156','E167','E210','E229','E237','E256']
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False,
                       exclude='bads')

In [143]:
raw.plot_sensors() #plotting the sensor map in 2d
raw.plot_sensors(kind='3d', ch_type='eeg', ch_groups='position') #sensor map in 3d

### Projection mapping

In [114]:
mne.set_eeg_reference(raw) #prevent MNE from setting reference automatically

Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but hasn't been applied yet. Use the .apply_proj() method function to apply projections.


(<RawEEGLAB  |  None, n_channels x n_times : 258 x 107996 (432.0 sec), ~213.1 MB, data loaded>,
 None)

### Power plot

In [81]:
raw.plot_psd(average=False)

tmax defaults to 60. in 0.14 but will change to np.inf in 0.15. Set it explicitly to avoid this warning


<ipython-input-81-0ad863b69bc9>:1: DeprecationWarning: tmax defaults to 60. in 0.14 but will change to np.inf in 0.15. Set it explicitly to avoid this warning
  raw.plot_psd(average=False)


Effective window size : 8.192 (s)


### Filtering of data

In [116]:
raw.filter(l_freq=None, h_freq=50) #low-pass filter at 50 Hz
raw.filter(l_freq=1, h_freq=None) #high-pass filter at 1 Hz
raw.notch_filter(freqs=[50,100], picks=picks, filter_length='auto', phase='zero')

Setting up low-pass filter at 50 Hz
h_trans_bandwidth chosen to be 12.5 Hz
Filter length of 132 samples (0.528 sec) selected
Setting up high-pass filter at 1 Hz
l_trans_bandwidth chosen to be 1.0 Hz
Filter length of 1650 samples (6.600 sec) selected
Setting up band-stop filter
Filter length of 3300 samples (13.200 sec) selected


<RawEEGLAB  |  None, n_channels x n_times : 258 x 107996 (432.0 sec), ~213.1 MB, data loaded>

In [84]:
raw.plot_psd(average=False)
raw.plot()

tmax defaults to 60. in 0.14 but will change to np.inf in 0.15. Set it explicitly to avoid this warning
Effective window size : 8.192 (s)


<ipython-input-84-3fa7306528c8>:1: DeprecationWarning: tmax defaults to 60. in 0.14 but will change to np.inf in 0.15. Set it explicitly to avoid this warning
  raw.plot_psd(average=False)


### Finding events

In [136]:
events = mne.find_events(raw, stim_channel='STI 014', shortest_event=1)
event_id = {'lett' : 110, 'aud_l' : 137, 'aud_r' : 138, 'go/nogo' : 131, 'vis' : 88}
print(event_id)

2818 events found
Events id: [ 41  42  43  44  45  46  51  52  53  54  55  56  81  82  83  84  85  86
  87  88 100 101 110 111 131 132 133 134 135 136 137 138]
{'aud_r': 138, 'vis': 88, 'lett': 110, 'aud_l': 137, 'go/nogo': 131}


### Event visualisation

In [122]:
# Plot the events to get an idea of the paradigm
# Specify colors
color = {131: 'green', 138: 'yellow', 137: 'red', 110: 'black', 88: 'c'}

mne.viz.plot_events(events, raw.info['sfreq'], raw.first_samp, color=color,
                    event_id=event_id)

event 41 missing from event_id will be ignored
event 42 missing from event_id will be ignored
event 43 missing from event_id will be ignored
event 44 missing from event_id will be ignored
event 45 missing from event_id will be ignored
event 46 missing from event_id will be ignored
event 51 missing from event_id will be ignored
event 52 missing from event_id will be ignored
event 53 missing from event_id will be ignored
event 54 missing from event_id will be ignored
event 55 missing from event_id will be ignored


<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 41 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 42 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 43 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 44 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 45 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 46 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 51 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 52 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: Runt

event 56 missing from event_id will be ignored
event 81 missing from event_id will be ignored
event 82 missing from event_id will be ignored
event 83 missing from event_id will be ignored
event 84 missing from event_id will be ignored
event 85 missing from event_id will be ignored
event 86 missing from event_id will be ignored
event 87 missing from event_id will be ignored
event 100 missing from event_id will be ignored
event 101 missing from event_id will be ignored
event 111 missing from event_id will be ignored


<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 56 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 81 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 82 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 83 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 84 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 85 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 86 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 87 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: Runt

event 132 missing from event_id will be ignored
event 133 missing from event_id will be ignored
event 134 missing from event_id will be ignored
event 135 missing from event_id will be ignored
event 136 missing from event_id will be ignored


<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 132 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 133 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 134 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 135 missing from event_id will be ignored
  event_id=event_id)
<ipython-input-122-40bd2d1cfa16>:6: RuntimeWarning: event 136 missing from event_id will be ignored
  event_id=event_id)


### Setting baseline

In [123]:
baseline = (None, 0)  # means from the first instant to t = 0

### Peak-to-peak rejection parameters

In [124]:
reject = dict(eeg=200e-6)

### Defining epochs and evoked responses for each trigger

### Visual trigger

In [125]:
  # event trigger and conditions
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, picks=picks,
                    baseline=baseline, preload=True, reject=reject)
epochs.drop_bad()
epochs_vis = epochs.pick_channels(['E9','E45','E186','E8','E17','E24',
                                    'E43','E52','E53','E80','E90','E81','E132','E157','E148',
                                  'E137','E125','E149','E116']) #Setting channels for 
                                            #evoked response of Visual trigger
evoked_vis = epochs_vis['vis'].average()
evoked_vis.plot(titles='Visual feedback evoked response', spatial_colors=True, gfp=True) #Plotting evoked response
                                                                                        # of visual channels
evoked_vis.plot_topomap()

1775 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1775 events and 176 original time points ...
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E25', u'E31', u'E32', u'E37', u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E174', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E71', u'E72', u'E75', u'E76', u'E83', u'E84', u'E85', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E148', u'E149', u'E150', u'E157', u'E158', u'E159', u'E160', u'E165', u'E166', u'E168', u'E169', u'E170', u'E174', u'E175', u'E177', u'E178', u'E187', u'E217', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E174', u'E247']
    Rejecting  epoch based on EEG : [u'E165', u'E166', u'E174']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u

    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E178', u'E199', u'E208', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E168', u'E174', u'E175', u'E178', u'E187', u'E199', u'E208', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187', u'E199', u'E251']
    Rejecting  epoch based on EEG : [u'E174', u'E178', u'E199', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E168', u'E174', u'E175', u'E187', u'E199', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E165', u'E166', u'E174', u'E178', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E165', u'E166', u'E168', u'E174', u'E178', u'E199', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E15', u'E16', u'E17', u'E18', u'E19', u'E23', u'E24', u'E25', u'E26', u'E30', u'E31', u'E32', u'E33', u'E36', u'E37', u'E3

    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E25', u'E37', u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E199', u'E208', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104'

    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E168', u'E175', u'E187', u'E233', u'E241', u'E242', u'E243', u'E244', u'E245', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E7', u'E8', u'E9', u'E10', u'E11', u'E16', u'E17', u'E18', u'E37', u'E43', u'E44', u'E45', u'E53', u'E60', u'E61', u'E62', u'E63', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130', u'E131', u'E132', u'E137', u'E138', u'E139', u'E140', u'E141', u'E142', u'E143', u'E144', u'E148', u'E149', u'E150', u'E151', u'E152', u'E153', u'E154', u'E155', u'E157', u'E158', u'E159', u'E160', u'E161', u

    Rejecting  epoch based on EEG : [u'E96', u'E106', u'E114', u'E165', u'E166', u'E174', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E165', u'E166', u'E174', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E165', u'E166', u'E174', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E165', u'E166', u'E174', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E165', u'E166', u'E174', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E96', u'E106', u'E114', u'E165', u'E166', u'E174', u'E175', u'E187', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E96', u'E106', u'E114', u'E165', u'E166', u'E174', u'E175', u'E187', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E165', u'E166', u'E174', u'E175', u'E187', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E165', u'E166', u'E174', u'E175', u'E187', u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E91', u'E165', u'

...

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130', u'E131',

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E18', u'E19', u'E20', u'E21', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E46', u'E47', u'E48', u'E49', u'E50', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E59', u'E60', u'E61', u'E62', u'E63', u'E67', u'E68', u'E69', u'E70', u'E73', u'E74', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E126', u'E127', u'E128', u'E129', u'E132', u'E137', u'E139', u'E140', u'E141', u'E142', u'E143', u'E144', u'E148', u'E149', u'E150', u'E151', u'E152', u'E153', u'E154', u'E155', u'E157', u'E161', u'E162', u'E163', u'E164', u'E

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E169', u'E170', u'E174', u'E177', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E32', u'E37', u'E96', u'E106', u'E114', u'E178', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E168', u'E174', u'E178', u'E187', u'E199', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51',

#### More visualisation

In [126]:
ts_args = dict(gfp=True)
topomap_args = dict(sensors=False)

evoked_vis.plot_joint(title='Visual evoked response', times = [.0, 0.075, 0.125, 0.25],
                      ts_args=ts_args, topomap_args=topomap_args)

### "Letter"  trigger

In [142]:
tmin = -0.5  # start of each epoch (200ms before the trigger)
tmax = 1  # end of each epoch (1000ms after the trigger)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, preload=True, reject=reject)
epochs.drop_bad()
epochs_lett = epochs.pick_channels(['E9','E45','E186','E8','E17','E24',
                                    'E43','E52','E53','E80','E90','E81','E132','E157','E148',
                                  'E137','E125','E149','E116'])
evoked_lett = epochs_lett['lett'].average()
evoked_lett.plot(titles='Button evoked response', spatial_colors=True, gfp=True)
evoked_lett.plot_topomap(title='Button evoked response power map over time')

1775 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1775 events and 376 original time points ...
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E70', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E165', u'E166', u'E168', u'E174', u'E175', u'E176', u'E178', u'E188', u'E200', u'E208', u'E228', u'E230', u'E231', u'E234', u'E235', u'E236', u'E238', u'E239', u'E240', u'E241', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E175', u'E187', u'E247']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E71', u'E72', u'E75', u'E76', u'E83', u'E84', u'E85', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E148', u'E149', u'E150', u'E157', u'E158', u'E159', u'E160', u'E165', u

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E187', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E104', u'E105', u'E165', u'E166', u'E174', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E38', u'E91', u'E92', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E91', u'E92', u'E96', u'E106', u'E114', u'E174', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E91', u'E92', u'E165', u'E166', u'E168', u'E174', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E91', u'E92', u'E165', u'E166', u'E168', u'E174', u'E178', u'E255']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E26', u'E31', u'E32'

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E37', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E148', u'E149', u'E150', u'E157', u'E165', u'E166', u'E168', u'E169', u'E170', u'E175', u'E176', u'E177', u'E178', u'E187', u'E188', u'E189', u'E199', u'E200', u'E208', u'E217', u'E233', u'E241', u'E251', u'E255']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E37', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E165', u'E166', u'E168', u'E169', u'E170', u'E175', u'E176', u'E177', u'E178', u'E187', u'E188', u'E189', u'E199', u'E200', u'E208', u'E217', u'E233', u'E234', u'E235', u'E238', u'E239', u'E240', u'E247', u'E251', u'E255']
    Rejecting  epoch based on EEG : [u'E3', u'E4', u'E10', u'E11', u'E12', u'E18', u'E19', u'E32', u'E33', u'E37', u'E38', u'E44', u'E69', u'E70', u'E74', u'E77', u'E83', u'E86', u'E87', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', 

...

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E38', u'E46', u'E83', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E150', u'E160', u'E165', u'E166', u'E168', u'E169', u'E170', u'E174', u'E177', u'E178', u'E199', u'E200', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E38', u'E46', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E169', u'E170', u'E174', u'E177', u'E178', u'E199', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E38', u'E46', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E169', u'E170', u'E174', u'E177', u'E178', u'E199', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch

1724 bad epochs dropped


In [141]:
evoked_lett.plot_joint(title='Button evoked response', times = [0, 0.5, 1],
                      ts_args=ts_args, topomap_args=topomap_args)

### Auditory/Left

In [152]:
tmin = -0.02  # start of each epoch (20ms before the trigger)
tmax = 0.075 # end of each epoch (75ms after the trigger)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, preload=True, reject=reject)
epochs_aud_l = epochs.pick_channels(['E43','E52','E53','E80','E90','E81','E251'
                                     'E255','E73','E92','E91','E254','E250','E67',
                                    'E68','E69','E74','E227','E218',
                                     'E216','E233','E209','E232','E219'])
evoked_aud_l = epochs_aud_l['aud_l'].average()
evoked_aud_l.plot(titles='Auditory/Left evoked response', spatial_colors=True, gfp=True)
evoked_aud_l.plot_topomap()

1775 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1775 events and 25 original time points ...
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E251']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E165', u'E166', u'E208', u'E247']
    Rejecting  epoch based on EEG : [u'E2', u'E3', u'E10', u'E11', u'E38', u'E41', u'E46', u'E47', u'E48', u'E49', u'E50', u'E55', u'E56', u'E83', u'E93', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E178', u'E199', u'E205', u'E208', u'E228', u'E232', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38'

    Rejecting  epoch based on EEG : [u'E174', u'E255']
    Rejecting  epoch based on EEG : [u'E174', u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E233']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E168']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on

    Rejecting  epoch based on EEG : [u'E2', u'E3', u'E10', u'E11', u'E41', u'E50', u'E195', u'E203', u'E204', u'E205', u'E212', u'E214', u'E222', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E2', u'E10', u'E11', u'E39', u'E40', u'E41', u'E46', u'E48', u'E49', u'E50', u'E56', u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E115', u'E149', u'E150', u'E168', u'E175', u'E187', u'E195', u'E200', u'E205', u'E208', u'E212', u'E220', u'E221', u'E232', u'E236', u'E240', u'E243', u'E246', u'E250', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E94', u'E115', u'E251']
    Rejecting  epoch based on EEG : [u'E168', u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E175', u'E176', u'E187', u'E188', u'E189', u'E199', u'E200', u'E208', 

    Rejecting  epoch based on EEG : [u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E55', u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E46', u'E56', u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E46', u'E56', u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E47', u'E221']
    Rejecting  epoch based on EEG : [u'E221', u'E232']
    Rejecting  epoch based on EEG : [u'E56', u'E221']
    Rejecting  epoch based on EEG : [u'E221', u'E232']
    Rejecting  epoch based on EEG : [u'E221']
    Rejecting  epoch based on EEG : [u'E221']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E221']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E221']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E25', u'E32', u'E37', u'E221', u'E222']
    Rejecting  epoch based on EEG : [u'E221']
  

    Rejecting  epoch based on EEG : [u'E83', u'E94']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting 

...

    Rejecting  epoch based on EEG : [u'E217']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E168']
    Rejecting  epoch based on EEG : [u'E168']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168']
    Rejecting  epoch based on EEG : [u'E168']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Reje

    Rejecting  epoch based on EEG : [u'E10', u'E39', u'E41', u'E50', u'E56', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E217', u'E221', u'E228', u'E233', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E221', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E221', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E130'

    Rejecting  epoch based on EEG : [u'E10', u'E39', u'E40', u'E44', u'E46', u'E47', u'E48', u'E49', u'E54', u'E55', u'E56', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E168', u'E175', u'E178', u'E187', u'E199', u'E205', u'E212', u'E217', u'E218', u'E223', u'E226', u'E228', u'E230', u'E232', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E10', u'E11', u'E18', u'E38', u'E39', u'E40', u'E41', u'E44', u'E48', u'E49', u'E50', u'E54', u'E55', u'E56', u'E58', u'E61', u'E65', u'E67', u'E68', u'E70', u'E73', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E175', u'E178', u'E187', u'E195', u'E196', u'E199', u'E203', u'E205', u'E206', u'E212', u'E213', u'E214', u'E217', u'E218', u'E219', u'E220', u'E222', u'E223', u'E224', u'E225', u'E227', u'E230', u'E231', u'E232', u'E233', u'E234', u'E235', u'E236', u'E243', u'E246', u'E250', u'E251']
    Rejecting  epoch base

    Rejecting  epoch based on EEG : [u'E1', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E21', u'E22', u'E23', u'E24', u'E29', u'E30', u'E34', u'E39', u'E41', u'E43', u'E44', u'E45', u'E50', u'E60', u'E70', u'E77', u'E79', u'E81', u'E83', u'E86', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E127', u'E128', u'E129', u'E130', u'E131', u'E132', u'E139', u'E140', u'E141', u'E144', u'E151', u'E152', u'E153', u'E154', u'E155', u'E158', u'E159', u'E160', u'E161', u'E162', u'E165', u'E166', u'E168', u'E171', u'E172', u'E173', u'E174', u'E175', u'E178', u'E179', u'E183', u'E185', u'E186', u'E187', u'E192', u'E193', u'E196', u'E198', u'E199', u'E202', u'E205', u'E206', u'E207', u'E211', u'E214', u'E215', u'E217', u'E219', u'E220', u'E221', u'

    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E115', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E115', u'E174', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E115', u'E168', u'E174', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E168', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E174', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u

In [138]:
evoked_aud_l.plot_joint(title='Auditory/Left evoked response', times = [0, 0.018, 0.033],
                      ts_args=ts_args, topomap_args=topomap_args)

### Auditory/Right

In [153]:
tmin = -0.02  # start of each epoch (20ms before the trigger)
tmax = 0.075  # end of each epoch (75ms after the trigger)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, preload=True, reject=reject) #need to redefine epochs each time
                                                                    #perhaps could be streamlined?
epochs_aud_r = epochs.pick_channels(['E43','E52','E53','E80','E90','E81','E251'
                                     'E255','E73','E92','E91','E254','E250','E67',
                                    'E68','E69','E74','E227','E218',
                                     'E216','E233','E209','E232','E219'])
evoked_aud_r = epochs_aud_r['aud_r'].average()
evoked_aud_r.plot(titles='Auditory/Right evoked response', spatial_colors=True, gfp=True)
evoked_aud_r.plot_topomap()

1775 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1775 events and 25 original time points ...
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E251']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E165', u'E166', u'E208', u'E247']
    Rejecting  epoch based on EEG : [u'E2', u'E3', u'E10', u'E11', u'E38', u'E41', u'E46', u'E47', u'E48', u'E49', u'E50', u'E55', u'E56', u'E83', u'E93', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E178', u'E199', u'E205', u'E208', u'E228', u'E232', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38'

    Rejecting  epoch based on EEG : [u'E174', u'E255']
    Rejecting  epoch based on EEG : [u'E174', u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E233']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E233', u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E168']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on EEG : [u'E255']
    Rejecting  epoch based on

    Rejecting  epoch based on EEG : [u'E2', u'E3', u'E10', u'E11', u'E41', u'E50', u'E195', u'E203', u'E204', u'E205', u'E212', u'E214', u'E222', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E2', u'E10', u'E11', u'E39', u'E40', u'E41', u'E46', u'E48', u'E49', u'E50', u'E56', u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E115', u'E149', u'E150', u'E168', u'E175', u'E187', u'E195', u'E200', u'E205', u'E208', u'E212', u'E220', u'E221', u'E232', u'E236', u'E240', u'E243', u'E246', u'E250', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E94', u'E115', u'E251']
    Rejecting  epoch based on EEG : [u'E168', u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E175', u'E176', u'E187', u'E188', u'E189', u'E199', u'E200', u'E208', 

    Rejecting  epoch based on EEG : [u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E55', u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E46', u'E56', u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E46', u'E56', u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E221', u'E232', u'E251']
    Rejecting  epoch based on EEG : [u'E47', u'E221']
    Rejecting  epoch based on EEG : [u'E221', u'E232']
    Rejecting  epoch based on EEG : [u'E56', u'E221']
    Rejecting  epoch based on EEG : [u'E221', u'E232']
    Rejecting  epoch based on EEG : [u'E221']
    Rejecting  epoch based on EEG : [u'E221']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E221']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E221']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E25', u'E32', u'E37', u'E221', u'E222']
    Rejecting  epoch based on EEG : [u'E221']
  

    Rejecting  epoch based on EEG : [u'E83', u'E94']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E104', u'E105']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting  epoch based on EEG : [u'E251']
    Rejecting 

...

    Rejecting  epoch based on EEG : [u'E217']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E168']
    Rejecting  epoch based on EEG : [u'E168']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E168']
    Rejecting  epoch based on EEG : [u'E168']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E165', u'E166']
    Rejecting  epoch based on EEG : [u'E168', u'E175', u'E187']
    Reje

    Rejecting  epoch based on EEG : [u'E10', u'E39', u'E41', u'E50', u'E56', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E217', u'E221', u'E228', u'E233', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E221', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E168', u'E174', u'E175', u'E178', u'E187', u'E221', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E130'

    Rejecting  epoch based on EEG : [u'E10', u'E39', u'E40', u'E44', u'E46', u'E47', u'E48', u'E49', u'E54', u'E55', u'E56', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E168', u'E175', u'E178', u'E187', u'E199', u'E205', u'E212', u'E217', u'E218', u'E223', u'E226', u'E228', u'E230', u'E232', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E10', u'E11', u'E18', u'E38', u'E39', u'E40', u'E41', u'E44', u'E48', u'E49', u'E50', u'E54', u'E55', u'E56', u'E58', u'E61', u'E65', u'E67', u'E68', u'E70', u'E73', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E175', u'E178', u'E187', u'E195', u'E196', u'E199', u'E203', u'E205', u'E206', u'E212', u'E213', u'E214', u'E217', u'E218', u'E219', u'E220', u'E222', u'E223', u'E224', u'E225', u'E227', u'E230', u'E231', u'E232', u'E233', u'E234', u'E235', u'E236', u'E243', u'E246', u'E250', u'E251']
    Rejecting  epoch base

    Rejecting  epoch based on EEG : [u'E1', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E21', u'E22', u'E23', u'E24', u'E29', u'E30', u'E34', u'E39', u'E41', u'E43', u'E44', u'E45', u'E50', u'E60', u'E70', u'E77', u'E79', u'E81', u'E83', u'E86', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E127', u'E128', u'E129', u'E130', u'E131', u'E132', u'E139', u'E140', u'E141', u'E144', u'E151', u'E152', u'E153', u'E154', u'E155', u'E158', u'E159', u'E160', u'E161', u'E162', u'E165', u'E166', u'E168', u'E171', u'E172', u'E173', u'E174', u'E175', u'E178', u'E179', u'E183', u'E185', u'E186', u'E187', u'E192', u'E193', u'E196', u'E198', u'E199', u'E202', u'E205', u'E206', u'E207', u'E211', u'E214', u'E215', u'E217', u'E219', u'E220', u'E221', u'

    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E115', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E115', u'E174', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E115', u'E168', u'E174', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E168', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E174', u'E199', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u

In [140]:
evoked_aud_r.plot_joint(title='Auditory/Right evoked response', times = [0, 0.018, 0.033],
                      ts_args=ts_args, topomap_args=topomap_args)

### Average of Auditory trials

In [154]:
left, right = epochs_aud_l["aud_l"].average(), epochs_aud_r["aud_r"].average()

# create and plot difference ERP
mne.combine_evoked([left, -right], weights='equal').plot_joint(times=[0,0.018,0.025,0.033])

### Go/nogo inhibition task

In [148]:
tmin = -0.5  # start of each epoch (200ms before the trigger)
tmax = 1  # end of each epoch (1000ms after the trigger)
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=baseline, preload=True, reject=reject)
epochs.drop_bad()
epochs_go = epochs.pick_channels(['E31','E32','E26','E25','E18','E19','E20','E27'
                                  'E33','E37','E9','E45','E186','E8','E17','E24',
                                'E43','E52','E53','E80','E90','E81','E132','E157','E148',
                                  'E137','E125','E149','E116'])
evoked_go = epochs_go['go/nogo'].average()
evoked_go.plot(titles='Go/nogo task evoked response', spatial_colors=True, gfp=True)
evoked_go.plot_topomap(title='Go/nogo task evoked response power map over time')

1775 matching events found
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1775 events and 376 original time points ...
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E70', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E165', u'E166', u'E168', u'E174', u'E175', u'E176', u'E178', u'E188', u'E200', u'E208', u'E228', u'E230', u'E231', u'E234', u'E235', u'E236', u'E238', u'E239', u'E240', u'E241', u'E247', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E83', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E175', u'E187', u'E247']
    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E37', u'E71', u'E72', u'E75', u'E76', u'E83', u'E84', u'E85', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E148', u'E149', u'E150', u'E157', u'E158', u'E159', u'E160', u'E165', u

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E10', u'E18', u'E83', u'E91', u'E92', u'E93', u'E94', u'E95', u'E104', u'E105', u'E165', u'E166', u'E168', u'E174', u'E187', u'E251']
    Rejecting  epoch based on EEG : [u'E83', u'E91', u'E92', u'E93', u'E94', u'E104', u'E105', u'E165', u'E166', u'E174', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E38', u'E91', u'E92', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E91', u'E92', u'E96', u'E106', u'E114', u'E174', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E37', u'E91', u'E92', u'E165', u'E166', u'E168', u'E174', u'E175', u'E187']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E91', u'E92', u'E165', u'E166', u'E168', u'E174', u'E178', u'E255']
    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E26', u'E31', u'E32'

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E37', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E148', u'E149', u'E150', u'E157', u'E165', u'E166', u'E168', u'E169', u'E170', u'E175', u'E176', u'E177', u'E178', u'E187', u'E188', u'E189', u'E199', u'E200', u'E208', u'E217', u'E233', u'E241', u'E251', u'E255']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E18', u'E19', u'E37', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E165', u'E166', u'E168', u'E169', u'E170', u'E175', u'E176', u'E177', u'E178', u'E187', u'E188', u'E189', u'E199', u'E200', u'E208', u'E217', u'E233', u'E234', u'E235', u'E238', u'E239', u'E240', u'E247', u'E251', u'E255']
    Rejecting  epoch based on EEG : [u'E3', u'E4', u'E10', u'E11', u'E12', u'E18', u'E19', u'E32', u'E33', u'E37', u'E38', u'E44', u'E69', u'E70', u'E74', u'E77', u'E83', u'E86', u'E87', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', 

...

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E1', u'E2', u'E3', u'E4', u'E5', u'E6', u'E7', u'E8', u'E9', u'E10', u'E11', u'E12', u'E13', u'E14', u'E15', u'E16', u'E17', u'E18', u'E19', u'E20', u'E21', u'E22', u'E23', u'E24', u'E25', u'E26', u'E27', u'E28', u'E29', u'E30', u'E31', u'E32', u'E33', u'E34', u'E35', u'E36', u'E37', u'E38', u'E39', u'E40', u'E41', u'E42', u'E43', u'E44', u'E45', u'E46', u'E47', u'E48', u'E49', u'E50', u'E51', u'E52', u'E53', u'E54', u'E55', u'E56', u'E57', u'E58', u'E59', u'E60', u'E61', u'E62', u'E63', u'E64', u'E65', u'E66', u'E67', u'E68', u'E69', u'E70', u'E71', u'E72', u'E73', u'E74', u'E75', u'E76', u'E77', u'E78', u'E79', u'E80', u'E81', u'E83', u'E84', u'E85', u'E86', u'E87', u'E88', u'E89', u'E90', u'E91', u'E92', u'E93', u'E94', u'E95', u'E96', u'E97', u'E98', u'E99', u'E100', u'E101', u'E104', u'E105', u'E106', u'E107', u'E108', u'E109', u'E110', u'E114', u'E115', u'E116', u'E117', u'E118', u'E119', u'E125', u'E126', u'E127', u'E128', u'E129', u'E130',

    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E38', u'E46', u'E83', u'E92', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E115', u'E150', u'E160', u'E165', u'E166', u'E168', u'E169', u'E170', u'E174', u'E177', u'E178', u'E199', u'E200', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E38', u'E46', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E169', u'E170', u'E174', u'E177', u'E178', u'E199', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch based on EEG : [u'E10', u'E11', u'E12', u'E18', u'E19', u'E25', u'E31', u'E32', u'E37', u'E38', u'E46', u'E83', u'E93', u'E94', u'E95', u'E96', u'E104', u'E105', u'E106', u'E114', u'E165', u'E166', u'E169', u'E170', u'E174', u'E177', u'E178', u'E199', u'E217', u'E228', u'E233', u'E251']
    Rejecting  epoch

1724 bad epochs dropped
